# Package
* Builds docker containers for all microservices

In [1]:
import NBInclude: @nbinclude
import JSON
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [2]:
function copy(file::String, dst::String)
    mkpath(joinpath(dst, dirname(file)))
    cp(file, joinpath(dst, file))
end

function build(app::String, basedir::String)
    name = split(app, "/")[end]
    run(`docker build -q -t rsys/$name $basedir/$name`)
end;

In [3]:
function fetch_media_lists(basedir::String)
    app = "$basedir/fetch_media_lists"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/fetch_media_lists", app)
    copy("notebooks/API/API", app)
    copy("environment/animeplanet", app)
    copy("environment/kitsu", app)
    copy("environment/mal", app)
    copy("environment/proxies", app)
    build(app, basedir)
end;

In [4]:
function compress_media_lists(basedir::String)
    app = "$basedir/compress_media_lists"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/compress_media_lists", app)
    copy("notebooks/ImportDatasets", app)
    copy("notebooks/ProcessData", app)
    copy("data/processed_data", app)
    build(app, basedir)
end;

In [5]:
function bagofwords_jl(basedir::String)
    app = "$basedir/bagofwords_jl"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/bagofwords_jl", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("notebooks/TrainingAlphas/Baseline/BaselineHelper.ipynb", "$app/App/src")
    for x in ALL_MEDIUMS
        copy("data/alphas/$x/Baseline/rating/params.jld2", "$app/App/src")
    end
    copy("data/processed_data", "$app/App/src")
    build(app, basedir)
end;

In [6]:
function bagofwords_py(basedir::String)
    for medium in ALL_MEDIUMS
        for metric in ALL_METRICS
            app = "$basedir/bagofwords_py_$(medium)_$(metric)"
            if ispath(app)
                rm(app; recursive = true)
            end
            cp("notebooks/Microservices/bagofwords_py", app)
            copy("notebooks/TrainingAlphas/BagOfWords/bagofwords.py", app)
            copy("data/alphas/$(medium)/BagOfWords/v1/$(metric)/model.pt", app)
            copy("data/alphas/$(medium)/BagOfWords/v1/$(metric)/config.json", app)
            build(app, basedir)
        end
    end
end;

In [7]:
function nondirectional(basedir::String)
    app = "$basedir/nondirectional"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/nondirectional", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("data/processed_data", "$app/App/src")
    for m in ALL_MEDIUMS
        copy("environment/alphas/$m/Nondirectional", "$app/App/src")
    end
    build(app, basedir)
end;

In [8]:
function transformer_jl(basedir::String)
    app = "$basedir/transformer_jl"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/transformer_jl", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("notebooks/TrainingAlphas/Transformer/Data.ipynb", "$app/App/src")
    copy("data/alphas/all/Transformer/v1/0/config.json", "$app/App/src")
    copy("data/processed_data", "$app/App/src")
    build(app, basedir)
end;

In [9]:
function transformer_py(basedir::String)
    for medium in ALL_MEDIUMS
        app = "$basedir/transformer_py_$(medium)"
        if ispath(app)
            rm(app; recursive = true)
        end
        cp("notebooks/Microservices/transformer_py", app)
        copy("notebooks/TrainingAlphas/Transformer/transformer.py", app)
        copy("data/alphas/$(medium)/Transformer/v1/model.pt", app)
        copy("data/alphas/all/Transformer/v1/0/config.json", app)
        build(app, basedir)
    end
end;

In [10]:
function ensemble(basedir::String)
    app = "$basedir/ensemble"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/ensemble", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("notebooks/TrainingAlphas/Ensemble/EnsembleInputs.ipynb", "$app/App/src")
    copy("data/processed_data", "$app/App/src")
    for m in ALL_MEDIUMS
        copy("environment/alphas/$m/Linear", "$app/App/src")
    end
    build(app, basedir)
end;

In [28]:
function get_endpoints()
    apps = [
        [
            "index",
            "fetch_media_lists",
            "compress_media_lists",
            "nondirectional",
            "bagofwords_jl",
            "transformer_jl",
            "ensemble",
        ]
        ["bagofwords_py_$(x)_$(y)" for x in ALL_MEDIUMS for y in ALL_METRICS]
        ["transformer_py_$(x)" for x in ALL_MEDIUMS]
    ]
    settings = get_settings()
    if get(settings, "cloud_hosting", false)
        domain = settings["cloud_hosting_domain"]
        tag = settings["cloud_hosting_tag"]
        d = Dict()
        for app in apps
            name = replace(app, "_" => "-")
            d[app] = "https://$(name)-$(tag)-$(domain)"
        end
        return d
    else
        return Dict(app => "http://$(app):8080" for app in apps)
    end
end

function index(basedir::String)
    app = "$basedir/index"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/index", app)
    open("environment/endpoints.json", "w") do f
        write(f, JSON.json(get_endpoints()))
    end
    copy("environment/endpoints.json", "$app/App/src")
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    build(app, basedir)
end;

In [12]:
function package(basedir::String)
    apps = [
        fetch_media_lists,
        compress_media_lists,
        bagofwords_jl,
        bagofwords_py,
        nondirectional,
        transformer_py,
        transformer_jl,
        ensemble,
        index,
    ]
    Threads.@threads for app in apps
        app(basedir)
    end
end

package (generic function with 1 method)

In [13]:
cd(joinpath(@__DIR__, "../.."))
basedir = "notebooks/Microservices/Images";

In [14]:
if ispath(basedir)
    rm(basedir; recursive = true)
end
mkpath(basedir)
run(`docker system prune -f`);

Deleted Images:
untagged: us-central1-docker.pkg.dev/rsys-417115/rsys-images/index@sha256:0a048d50dff4688b4206903908e5294cf9a0ca86b791f18085408e83bf6c9676
deleted: sha256:776d1e3166e30722a2e8e488d2534c5f8e460c44d00572d6ac6ddca14c98feea

Deleted build cache objects:
wx8cx42ru87obcmmjptiabw9l
loyku5xlar497m07goc1acbrc
4zvr3k1enu1l272kjl8711kgl
jgnhmeoomv0xcpmpqqkn5camh
o8eueie8li3iwswe5m9oc1w3t
nma10xsyl82fqs1nuiwuo8q8m
j425h974gbxign1g9le5hlhnj
ngm4cxoksjteh16fm4kbpn4ia
vo9y9ae8l1y3v6bve31d1cmy4
vhh8u502gttmbclxjj5lcewy7
lx404dgtrn2glpnrk6zfbbcbg
vc2lkk85doureun4ashpn4yl4
g8yhcu733ha2ap83p9omfzr0j
114nvxguygbetyb97bylbjftc
xrpm9xkjbl7bgbioue0td0rtx
k9isi3fryj99ugpmtb9stdedh
e544czfpf735a877mq2aprvts
s85wcxwpmcu3dq464eowpvjxl
aurfna4v2ermbir1p64f8m333
wz7m0nqtg6zdn3djk8t8tjcxq
37auo7as2vpv64xdcwje47j08
tvbkjkpu51eobday7wv8e7o73
oz24dz4xv2lb7b0l6pwamb10f
qr0dbpyfhea4fv5manfi4e660
mt5fit1xnbvkgcisar9qrbbdh
ymu65yorxlqvnpif83tayyfg8
bahc7skx97iybb2pw6yv66sch
uop6izcygwax8sapk7u1dge47
w39kfw

In [15]:
package(basedir);

sha256:628f7bda2326e8032638e843e93d5bd31cd618bb5f3e06702d1b2a6464cd7493
sha256:1a1794523276bd4f78b00039c3d41628e481e75890a1db3df9ce851e47bf841d
sha256:01b882f242bfe75f10c006d4405b99cee4692ce13473476cfe0e3570ab86db7b
sha256:054fb5974f28a26737951ac9cfce75343598b6853c3316a070f349d083eb966c
sha256:9a4658fbc7794546024201f407d2aa8c4dfd5e4b2200c639dbc55310b2bae8d7
sha256:dd6dfbbb217029d6474fcf84904e0cac422ebab205ae5072969d44aa3d4ff11b
sha256:730f99959e2894e13439f0f106fe1fc5deed59c9e57f20f2b3a63e6284f63464
sha256:0d844326f3a9f0b5e9aeae97d1e760981a4c299dcfc59a7a303ef1f4bb8071ae
sha256:afc48b928528a16e5cdb04482c9776ffae86df0cf0cf2c749f2f339bb8acee57
sha256:141c6f902540f7a8fb9b3d997c12dbe2a90993b95563b576067f24f193e52fa6
sha256:6b0d4f6a8398f9b245e531fab196a33ae3c2e54dcf2e0e925e571c766087ad43
sha256:033a775aa9d89742f11e44656b77b44d4005fb2293df75cc0a0c750e1aa43bbb
sha256:b76c208680ab389872e056e4d2d8df91fafe09809c45cff06e4a8b46f7bf5b54
sha256:9ba2ba024b578c805a3505672735e6f0cf29b07032c4fc7e14820eb70

In [36]:
index(basedir);

sha256:7e6dce63b51172a318a7a5712074a7448550c5314963becd561b29215fb13e96


In [40]:
index(basedir);

sha256:6421e17fe2dcf5224b5d63535bc11345fdfac706987953f920faeaf4f5a154b2
